In [ ]:
import requests
import pandas as pd

In [ ]:
base_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-france-commune/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
response = requests.get(base_url)
response.raise_for_status()

In [ ]:
filename = "georef-france-commune.csv"
with open(filename, "wb") as f:
    f.write(response.content)
print(f"Downloaded {filename}")

In [ ]:
data.columns

In [ ]:
data = pd.read_csv(
    filename,
    delimiter=";",
    usecols=[
        "Geo Point",
        "Nom Officiel Département",
        "Nom Officiel Commune",
        # "Code Canton Ville",
    ],
)

In [ ]:
data.to_parquet("georef-france-commune.parquet")

In [ ]:
data.head()

In [ ]:
data.query("`Nom Officiel Commune` == 'Craponne'")

In [ ]:
data.to_csv("georef-france-commune.csv", index=False)

# Reindex from grid points

In [ ]:
from pollen_forecast.copernicus import PollenForcastCopernicusGeneric

In [ ]:
my_pollen = PollenForcastCopernicusGeneric(
    north=51.70, south=41.87, east=8.74, west=-5.33, prefix="./france_territory/"
)
my_pollen.get_pollen_data()

In [ ]:
import xarray as xr

data = xr.open_dataset(my_pollen.filename)
data.coords["longitude"] = (data.coords["longitude"] + 180) % 360 - 180
data_sorted = data.sortby("longitude")
data_sorted

In [ ]:
import cartopy
from cartopy import crs as ccrs
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
data_sorted["gpg_conc"].isel(time=0, level=0).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
)

ax.coastlines()
# add borders
ax.add_feature(cartopy.feature.BORDERS, linestyle=":")

In [ ]:
communes = pd.read_csv("georef-france-commune.csv")
communes.head()

In [ ]:
# find the neasrest grip point of the Xarray Dataset for each commune
def find_nearest_grip_point(lat, long, data):
    print(f"Finding nearest grip point for {lat}, {long}")
    point = data.sel(latitude=lat, longitude=long, method="nearest")
    try:
        lat_nearest = point.latitude.values
        long_nearest = point.longitude.values
    except IndexError:
        print(f"Error with {lat}, {long}")
        print(point.latitude.values)
        print(point.longitude.values)
        return None, None

    return lat_nearest, long_nearest


communes["latitude"], communes["longitude"] = zip(
    *communes.apply(
        lambda x: find_nearest_grip_point(
            x["Geo Point"].split(",")[0], x["Geo Point"].split(",")[1], data_sorted
        ),
        axis=1,
    )
)
communes.head()

In [ ]:
communes.to_csv("georef-france-commune.csv", index=False)

In [ ]:
ax = (
    data_sorted["gpg_conc"]
    .sel(level=0, longitude=2.35, latitude=48.85, method="nearest")
    .plot()
)

In [ ]:
data_sorted["gpg_conc"].sel(
    level=0, latitude=4.85, longitude=45.75, method="nearest"
).plot()